# Quick Start Guide

This guide shows you how to set up and use *scarlet2* to model a hyperspectral image cube. We assume that you have a full installation of _scarlet2_ including `optax`, `numpyro`, and `h5py`. We'll see below where these packages are needed.

In [ ]:
# Import Packages and setup
import jax.numpy as jnp
import matplotlib.pyplot as plt

from scarlet2 import *

## Load the Data

We load an example data set (here an image cube from HSC with 5 bands) *and* a detection catalog. If such a catalog is not available, packages like [SEP](http://sep.readthedocs.io/) and [photutils](https://photutils.readthedocs.io/en/stable/) will happily generate one.

To make tests like this one convenient, we have a separate HuggingFace data set. To access it, you need to `pip install huggingface_hub`:

In [ ]:
from huggingface_hub import hf_hub_download

filename = hf_hub_download(
    repo_id="astro-data-lab/scarlet-test-data", filename="hsc_cosmos_35.npz", repo_type="dataset"
)
file = jnp.load(filename)
data = jnp.asarray(file["images"])
channels = [str(f) for f in file["filters"]]
centers = jnp.array([(src["y"], src["x"]) for src in file["catalog"]])  # Note: y/x convention!
weights = jnp.asarray(1 / file["variance"])
psf = jnp.asarray(file["psfs"])

```{warning}

Coordinates in *scarlet* are given in the C/numpy notation (y,x) as opposed to the more conventional (x,y) ordering.
```

## Define Observation and Frame

An {py:class}`~scarlet2.Observation` combines several data elements with a {py:class}`~scarlet2.Frame`, similar to header-data arrangement in FITS files. In addition to the actual science image cube, you can and often must provide weights for all elements in the data cube, an image cube of the PSF model (one image for all or one for each channel), an {py:class}`astropy.wcs.WCS` structure to translate from pixel to sky coordinates, and labels for all channels.

The {py:class}`~scarlet2.Frame` is a description of the hyperspectral cube of the observations. Think of it as the metadata, what aspects of the sky are described here. At the least, a {py:class}`~scarlet2.Frame` holds the `shape` of the cube, for which we use the convention `(C, H, W)` for the number of elements in 3 dimensions: `C` for the number of bands/channels and `H/W` for the height and width of the images.





In [ ]:
obs = Observation(
    data,
    weights,
    psf=ArrayPSF(psf),
    channels=channels,
)
model_frame = Frame.from_observations(obs)

_scarlet2_ will construct another {py:class}`~scarlet2.Frame` for the _model_ of the sky you seek to fit. It differs from the observation frame because the model needs to be superior in terms of spatial resolution, PSF blurring, spectral coverage, or all of the above, so that the observation can be computed from the model by a degradation operation, the so-called "forward operator. In _scarlet2_, {py:class}`~scarlet2.renderer.Renderer` implements this operation and contains, e.g., PSF difference kernels and resampling transformations.

In this example, we assume that bands and pixel locations are identical between the model and the observation. Because we have ground-based images, the degradation comes from the PSF. With different PSFs in each band, we need to define a reference PSF that is sufficiently narrow. Our default is minimal Gaussian PSF that is barely well-sampled (standard deviation of 0.7 pixels) as our reference kernel:

Note also the validation results that are automatically generated.
These are executed automatically on behalf of the user and act as guardrails.
Here we can see that check number ``007`` produced a error and prints a message indicating that the PSF is not the same for all channels.
While validation errors will not cause the code to stop running, they do indicate a likely problem that warrants attention.
Automatic validation is performed at several times in ``scarlet2``, which can be seen later in this notebook.
For additional information about automatic validation, please see the ["Validation" section](3-validation) of the documentation.

In [ ]:
print(model_frame)

If other properties of the model are desired, they can be provided to {py:func}`scarlet2.Frame.from_observations`.

## Display Image Cube

We can now make use of the plotting function {py:func}`scarlet2.plot.observation` to create a RGB image of our data.
We're going to use the {py:class}`~scarlet2.plot.AsinhAutomaticNorm`, which scales the observed data by a {math}`\mathrm{sinh}^{-1}` function that is automatically tuned to reveal both the noise level and the highlights, to create a color-consistent RGB image.

In [ ]:
norm = plot.AsinhAutomaticNorm(obs)
plot.observation(obs, norm=norm, sky_coords=centers, show_psf=True, add_labels=True)
plt.show()

Since we use no `wcs` in this {py:class}`~scarlet2.Observation`, all coordinates are already in image pixels, otherwise RA/Dec pairs are expected as `astropy` sky coordinates.

## Initialize Sources

You now need to define sources that are going to be fit. The full model, which we will call {py:class}`~scarlet2.Scene`, is a collection of {py:class}`~scarlet2.Source`s. Each source contains at least one {py:class}`~scarlet2.Component`, and each of those is defined by a `center`, `spectrum`, and `morphology`. You can have as many sources in a scene, or as many components in a source, as you want. To represent stars or other point sources, we provide the class {py:class}`~scarlet2.PointSource`, which adopts the PSF of the model frame as the morphological model of the source.

To adding sources to a scene is easy. You create the scene (as a python context with the `with` keyword) and then create a source inside of that context. It will automatically be added to the overall scene model. But now we have to decide what the initial values of `spectrum` and `morphology` should be. You can choose them as you see fit, but we also provide convenience methods. For instance {py:func}`scarlet2.init.pixel_spectrum` takes the spectrum of one pixel (typically the center pixel), and {py:func}`scarlet2.init.from_gaussian_moments` measures the spectrum and morphology by measuring the 2nd moments and creating an elliptical Gaussian to match those.

In [ ]:
with Scene(model_frame) as scene:
    for center in centers:
        try:
            spectrum, morph = init.from_gaussian_moments(obs, center, min_corr=0.99)
        except ValueError:
            spectrum = init.pixel_spectrum(obs, center)
            morph = init.compact_morphology()

        Source(center, spectrum, morph)

If we know something about the scene, we might want to customize the modeling. Let's assume that we know that object 0 is a star. We could just replace that source with a {py:class}`~scarlet2.PointSource`, but for clarity we rebuild the entire source list, making one change for source 0 and accepting everything else from the default initialization above:

In [ ]:
with Scene(model_frame) as scene:
    for i, center in enumerate(centers):
        if i == 0:  # we know source 0 is a star
            spectrum = init.pixel_spectrum(obs, center, correct_psf=True)
            PointSource(center, spectrum)
        else:
            try:
                spectrum, morph = init.from_gaussian_moments(obs, center, min_corr=0.99)
            except ValueError:
                spectrum = init.pixel_spectrum(obs, center)
                morph = init.compact_morphology()

            Source(center, spectrum, morph)

We can now check what we have in `scene`:

In [ ]:
print(scene)

`scene` contains the `frame` and the list `sources`. The first source is indeed a {py:class}`~scarlet2.PointSource`, which has a `morphology` model, namely the same {py:class}`~scarlet2.GaussianMorphology` as the PSF of the model frame.
 All others are standard {py:class}`~scarlet2.Source`s. The data portions in these models are listed as, e.g., `f32[11,11]`, which denotes an image array of 11x11 pixels. The size of these boxes was determined by {py:func}`scarlet2.init.from_gaussian_moments` to contain most of the flux (or the largest box that seems to be occupied by only one source).

## Fit the Model

The {py:class}`~scarlet2.Scene` class holds the list of sources and creates a model of the sky that (like any {py:class}`~scarlet2.Module`) can be evaluated, which allows us to determine the log-likelihood of the data given the model:

In [ ]:
scene_array = scene()  # evaluate the model
print("Initial likelihood:", obs.log_likelihood(scene_array))

But as most of us don't think in terms of likelihoods, we can also compute the goodness of fit of the model, which is identical to the log-likelihood up to a constant but much easier to interpret: It's the average chi^2 over all pixels with non-zero weights.

In [ ]:
print("Avg. chi^2:", obs.goodness_of_fit(scene_array))

You can now write your own fitter or sampler, but we provide the machinery for that as well. We're using `optax` for the optimization. First we need to define which {py:class}`~scarlet2.Parameters` of the model should be optimized. For each {py:class}`~scarlet2.Parameter`, that includes a stepsize, and, optionally, differentiable constraints from `numpyro`. Below is our recommended configuration:

In [ ]:
from functools import partial
from numpyro.distributions import constraints

# best step size for parameters with a relative "uncertainty":
# big/bright sources adjust more quickly
spec_step = partial(relative_step, factor=0.05)
morph_step = partial(relative_step, factor=1e-3)

with Parameters(scene) as parameters:
    for i in range(len(scene.sources)):
        Parameter(
            scene.sources[i].spectrum,
            name=f"spectrum:{i}",
            constraint=constraints.positive,
            stepsize=spec_step,
        )
        if i == 0:
            Parameter(scene.sources[i].center, name=f"center:{i}", stepsize=0.1)
        else:
            Parameter(
                scene.sources[i].morphology,
                name=f"morph:{i}",
                constraint=constraints.unit_interval,
                stepsize=morph_step,
            )

A model parameter that is not listed in `parameters` will not be updated. Now we can run the fitting method, and update every one of the `parameters`. We give ourselves a head-start by first running a linear solver for the spectra given the shapes of every source:

In [ ]:
maxiter = 1000
scene_ = scene.fit(obs, parameters, max_iter=maxiter, e_rel=1e-4, progress_bar=True)

## Interact with Results

The updated source models are now available in `scene_`. We can evaluate the entire scene and compute the now much improved goodness of fit:

In [ ]:
scene_array = scene_()
print("Optimized chi^2:", obs.goodness_of_fit(scene_array))

### Display Full Scene

In [ ]:
plot.scene(
    scene_,
    obs,
    norm=norm,
    show_model=True,
    show_rendered=True,
    show_observed=True,
    show_residual=True,
    add_boxes=True,
)
plt.show()

The fit is overall quite good, with low residuals, but it's not perfect. Let's investigate...

### Access All Sources

Individual sources can be accessed and evaluated as well, but we have to decide in which frame we want to see them. There are three options:


In [ ]:
# Get models for source 1
source = scene_.sources[1]
# in its own source box (indicated by the white boxes in the figure above)
source_array = source()
# inserted in model frame
source_in_scene_array = scene_.evaluate_source(source)
# as it appears in observed frame
source_as_seen = obs.render(source_in_scene_array)

import matplotlib

# for imshow: don't interpolate the pixels, put origin to bottom
matplotlib.rc("image", interpolation="none", origin="lower")

fig, axes = plt.subplots(1, 3, figsize=(10, 4))
axes[0].imshow(plot.img_to_rgb(source_array, norm=norm))
axes[1].imshow(plot.img_to_rgb(source_in_scene_array, norm=norm))
axes[2].imshow(plot.img_to_rgb(source_as_seen, norm=norm))
plt.show()

We also provide a function to create all of these source images:

In [ ]:
plot.sources(
    scene_,
    norm=norm,
    observation=obs,
    show_model=True,
    show_rendered=True,
    show_observed=True,
    show_spectrum=True,
    add_markers=False,
    add_boxes=True,
)
plt.show()

We can see that each source "lives" in a smaller box and is then placed into the larger scene. The model of object 0 assumes the simple Gaussian shape of the model PSF, which is the internal representation of a point source.

It's noticeable that all other sources, however, show pixel patterns that don't seem right. Some of them follow other sources (especially source 1) or noise (sources 4-6). What we see here is the limitation of non-parametric models: they can fit anything, but the likelihood may not have enough information to constrain all of these degrees of freedom. More/stronger constraints or priors can help, and we will describe their use in [a separate tutorial](howto/priors).

### Source Measurements

As shown above, source models are generated in the model frame (dah...), from which any measurement can directly be made without having to deal with noise, PSF convolution, overlapping sources, etc.

For instance, the color information in these plots stems from the spectrum, i.e. the per-band amplitude, which are computed from the hyperspectral model by integrating over the morphology. The source spectra plots in the right panels above have done exactly that. The convention of these fluxes is given by the units and ordering of the original data cube.

In [ ]:
print("----------------- {}".format(channels))
for k, src in enumerate(scene_.sources):
    print("Source {}, Fluxes: {}".format(k, measure.flux(src)))

Other measurements (e.g. `centroid`) or {py:class}`~scarlet2.measure.Moments` are also implemented:

In [ ]:
g = measure.Moments(scene_.sources[1])
print("Source 1 ellipticity:", g.ellipticity)

All of these measurements are ordered by channel, so the ellipticity above is listed as `[[e1_g, e1_r, e1_i, e1_z, e1_y],[e2_g, e2_r, e2_i, e2_z, e2_y]]`. That they are all the same should not come as a surprise: For single-component models, there is no variation of the morphology across the channels.

### Save and Re-Use Model

To preserve the model for posterity, individual sources (or their sub-models) or entire scenes can be serialized into a HDF5 file:

In [ ]:
import scarlet2.io

id = 35
filename = "hsc_cosmos.h5"
scarlet2.io.model_to_h5(scene_, filename, id=id, overwrite=True)

The stored model be loaded in the same way. Every source can be utilized as before.

In [ ]:
scene__ = scarlet2.io.model_from_h5(filename, id=id)
plot.scene(scene__, observation=obs, norm=norm, add_boxes=True)
plt.show()

We will now add two more sources to account for the largest residuals we have seen above. As we don't know their location accurately, we allow the fitter to shift/recenter the sources.

In [ ]:
# enter the context of the scene to create new elements
with scene__:
    # add two marginally detected sources at their approximate locations
    yx = [(14.0, 44.0), (42.0, 9.0)]
    for center in yx:
        center = jnp.array(center)
        spectrum = init.pixel_spectrum(obs, center)
        morph = init.compact_morphology()
        Source(center, spectrum, morph)

# need to remake the parameter structure because we have more free parameters now
with Parameters(scene__) as parameters__:
    for i in range(len(scene__.sources)):
        Parameter(
            scene__.sources[i].spectrum,
            name=f"spectrum:{i}",
            constraint=constraints.positive,
            stepsize=spec_step,
        )
        if i == 0:
            Parameter(scene__.sources[i].center, name=f"center:{i}", stepsize=0.1)
        else:
            Parameter(
                scene__.sources[i].morphology,
                name=f"morph:{i}",
                constraint=constraints.unit_interval,
                stepsize=0.1,
            )

scene___ = scene__.fit(obs, parameters__, max_iter=maxiter, e_rel=1e-4, progress_bar=True)
print("New chi^2:", obs.goodness_of_fit(scene___()))

We can see that the fit is slightly better than before. Let's have a look at the new model:

In [ ]:
plot.scene(
    scene___,
    obs,
    norm=norm,
    show_model=True,
    show_rendered=True,
    show_observed=True,
    show_residual=True,
    add_boxes=True,
)
plt.show()

We can see the two new sources 7 and 8, and that most of the other features are very similar to before. As expected, the residuals have visibly improved where we added the source and are now dominated by a red-blue pattern in the center of source 1, which we should probably fit with 2 components.

Notice in plot on the right, source 0 has a slight color dipole close to the center.
If you'll recall when we created the Observation object a validation check produced an error that indicated misaligned PSF centroids.
That PSF centroid misalignment is the likely cause of the dipole.